Função para carregamento da tabela (por quarters)

In [0]:
def obter_tabela(tabela, trimestres):
    list_df=[]
    for trimestre in trimestres:
        ano = trimestre[:4]
        quarter = trimestre[-1]
        path = f'/FileStore/faers-downloaded/faers_ascii_{trimestre}/ASCII/{tabela}{ano[-2:]}Q{quarter}.txt'
        try:
            df = spark.read.csv(path, sep='$', header=True, inferSchema=True)
            list_df.append(df)
            print(f"Arquivo carregado com sucesso: {path}")
        except Exception as e:
            print(f"Erro ao carregar {path}: {str(e)}")
    if list_df:
        df_all = list_df[0]
        for df in list_df[1:]:
            df_all = df_all.unionAll(df)
        print("Todos os dataframes foram unidos com sucesso.")
        print(f"Número total de linhas: {df_all.count()}")
    else:
        print("Nenhum arquivo foi carregado com sucesso.")
    return df_all

Função para transformar campos de datas de int para datas

In [0]:
def convert_date(df,date_col,dest_col):
  # Suponha que o nome da sua coluna seja 'data_numero'
  df = df.withColumn("data_string", col(date_col).cast("string"))

  df = df.withColumn(dest_col, 
      when(length(col("data_string")) == 8, to_date(col("data_string"), "yyyyMMdd"))
      .when(length(col("data_string")) == 6, to_date(expr("concat(data_string, '01')"), "yyyyMMdd"))
      .when(length(col("data_string")) == 4, to_date(expr("concat(data_string, '0101')"), "yyyyMMdd"))
  )
  return df.drop(col("data_string"))

Função que restaura os metadados de um schema original

In [0]:
def restore_metadata(schema,df):
    for field in schema.fields:
        if field.name in df.columns:
            df = df.withMetadata(field.name, field.metadata)
    return df


Função que carrega parquets finais

In [0]:
def load_parquet(tabela):
    path = f'dbfs:/FileStore/FAERS-grupo-4/{tabela}_final'
    return spark.read.parquet(path)

DataFrame[occr_country: string, occr_country_text: string, reporter_country: string, reporter_country_text: string, occp_cod: string, occp_txt: string, rept_cod: string, rept_typ_txt: string, primaryid: bigint, caseid: bigint, caseversion: int, event_dt: date, mfr_dt: date, init_fda_dt: date, fda_dt: date, auth_num: string, mfr_num: string, mfr_sndr: string, lit_ref: string, age: int, age_cod: string, age_grp: string, sex: string, e_sub: boolean, wt: double, wt_cod: string, rept_dt: date, to_mfr: boolean, folup_rpt: boolean, age_in_years: double, wt_in_kg: double]